In [1]:
import numpy as np
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.version.cuda)         # Should print your CUDA version, e.g


True
12.1


In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import os
from pathlib import Path
import pandas as pd
import seaborn as sns
import torch
from torch_geometric.data import Data
from torch_geometric import utils
from environment_manager import *
name = "240318-initial parsing"
env = load_create_environment(name)
GHComponentTable.initialise()

Setting environment variables
Copying vanilla components
File copied successfully from C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240318-VanillaComponents\vanilla_components.csv to ExtractionEnvironments\240318-initial parsing\00-VanillaComponents\vanilla_components.csv.
Copying components
Copying gh files


In [3]:
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.colors as mcolors

def plot_graph_by_category(graph):
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))

    # Get the unique categories
    categories = set(nx.get_node_attributes(graph, 'category').values())

    # Create a color map based on the number of categories
    num_colors = len(categories)
    color_map = plt.cm.get_cmap('viridis', num_colors)

    # Create a dictionary mapping categories to colors
    category_colors = {}
    for i, category in enumerate(categories):
        category_colors[category] = mcolors.to_hex(color_map(i))

    # Create a dictionary of node positions
    pos = nx.get_node_attributes(graph, 'pos')
    if not pos:  # If 'pos' attribute is not available, use 'x' and 'y' attributes
        pos = {node_id: (data['x'], data['y']) for node_id, data in graph.nodes(data=True)}

    # Create a list of colors based on the node categories
    node_colors = [category_colors[graph.nodes[node].get('category', '')] for node in graph.nodes()]

    # Draw the graph
    nx.draw(graph, pos=pos, with_labels=False, node_size=10, node_color=node_colors, edge_color='gray', arrows=True, ax=ax)

    # Create a legend
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=category,
                                  markerfacecolor=color, markersize=8)
                       for category, color in category_colors.items()]
    ax.legend(handles=legend_elements, title='Categories', loc='upper right')

    # Show the plot
    plt.axis('equal')
    plt.show()

In [4]:
import os
import networkx as nx

graph_folder = r'C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\ExtractionEnvironments\240318-initial parsing\05-GraphML'
# Initialize an empty list to store dictionaries
data_list = []


# Create a new graph to store the combined graph
combined_graph = nx.DiGraph()
graph_data = []
for i, file in enumerate(os.listdir(graph_folder)):
    if file.endswith('.graphml'):
        G = nx.read_graphml(os.path.join(graph_folder, file))
        # Skip files with no nodes to avoid division by zero
        if G.number_of_nodes() < 5:
            continue
        if G.number_of_edges() == 0:
            continue
        graph_data.append(G)
        combined_graph.add_nodes_from(G.nodes(data=True))
        combined_graph.add_edges_from(G.edges(data=True))

    

In [5]:
print(graph_data[0].nodes(data=True))


[('c9cde350-c7de-4df6-8a70-82cc57fd78ac', {'x': 428.0, 'y': 193.0, 'compid': '807b86e3-be8d-4970-92b5-f8cdcb45b06b', 'category': 'Curve'}), ('8f48460a-8768-456a-a63e-e1183c8c2f9e', {'x': 591.0, 'y': 217.0, 'compid': '962034e9-cc27-4394-afc4-5c16e3447cf9', 'category': 'Surface'}), ('1761219f-05c6-480d-a4f6-adfa5b5c8ea5', {'x': 139.5, 'y': 201.0, 'compid': '57da07bd-ecab-415d-9d86-af36d7073abc', 'category': 'Params'}), ('bb87905f-0d55-44fe-aea0-c50778c90631', {'x': 763.0, 'y': 250.0, 'compid': '5106bafc-d5d4-4983-83e7-7be3ed07f502', 'category': 'Surface'}), ('8b71cf8a-5e29-4ea0-8efe-04b349b8ad46', {'x': 435.0, 'y': 267.0, 'compid': '9103c240-a6a9-4223-9b42-dbd19bf38e2b', 'category': 'Vector'}), ('1087a0c7-90b1-41cd-88ed-22b3ec5cac33', {'x': 176.3834991455078, 'y': 271.78900146484375, 'compid': '57da07bd-ecab-415d-9d86-af36d7073abc', 'category': 'Params'}), ('f1c9c47f-4fd6-4aa7-911e-301230895ad6', {'x': 892.0, 'y': 215.0, 'compid': '41aa4112-9c9b-42f4-847e-503b9d90e4c7', 'category': 'Sets

In [6]:
for node, data in combined_graph.nodes(data=True):
    # Assign the values of 'x' and 'y' to 'posx' and 'posy'
    data['posx'] = data['x']
    data['posy'] = data['y']
    # Optionally, remove the old 'x' and 'y' keys if they're no longer needed
    del data['x']
    del data['y']

In [7]:
d = utils.from_networkx(combined_graph)
d

Data(edge_index=[2, 132214], compid=[110463], category=[110463], posx=[110463], posy=[110463], distance=[132214], s_paramidx=[132214], r_paramidx=[132214], s_access=[132214], r_access=[132214], s_datamapping=[132214], r_datamapping=[132214], s_is_optional=[132214], r_is_optional=[132214], num_nodes=110463)

In [8]:
# # Create a lookup table for all the nodes
# """we need to reorder the graphs because the node indices are not unique across the graphs and pytorch will order their indicies according to each node index"""
# node_lookup = {}
# node_counter = 0
# for i, graph in enumerate(graph_data):
#     for node in graph.nodes():
#         if node not in node_lookup:
#             node_lookup[node] = node_counter
#             node_counter += 1
# 
# reordered_graphs = []
# for graph in graph_data:
#     reordered_graph = nx.relabel_nodes(graph, {node: node_lookup[node] for node in graph.nodes()})
#     reordered_graphs.append(reordered_graph)



In [9]:
#remove the isolated nodes from the combined graph
combined_graph.remove_nodes_from(list(nx.isolates(combined_graph)))
combined_graph
# ds = utils.from_networkx(combined_graph)
# print(f'Number of nodes: {d.num_nodes}')
# print(f'Number of edges: {d.num_edges}')
# print(f'Average node degree: {d.num_edges / d.num_nodes:.2f}')
# print(f'Contains isolated nodes: {d.contains_isolated_nodes()}')
# print(f'Contains self-loops: {d.contains_self_loops()}')
# print(f'Is undirected: {d.is_undirected()}')


In [10]:
degs = sorted([d for n, d in list(nx.degree(combined_graph))], reverse=True)
print(f'Highest node degree: {degs[0]}')
print(f'Lowest node degree: {degs[-1]}')
print(f'Median node degree: {degs[len(degs) // 2]}')
print(f'Mean node degree: {sum(degs) / len(degs):.2f}')

Highest node degree: 53
Lowest node degree: 1
Median node degree: 2
Mean node degree: 2.48


In [11]:
ds = utils.from_networkx(combined_graph)
ds

Data(edge_index=[2, 132214], compid=[106450], category=[106450], posx=[106450], posy=[106450], distance=[132214], s_paramidx=[132214], r_paramidx=[132214], s_access=[132214], r_access=[132214], s_datamapping=[132214], r_datamapping=[132214], s_is_optional=[132214], r_is_optional=[132214], num_nodes=106450)

In [12]:
def get_component_name(id):
    p = GHComponentTable.search_component_by_guid(System.Guid(id))
    return f"{p.Desc.Category}:{p.Desc.Name}"

In [13]:
G = combined_graph
component_set = sorted(set(nx.get_node_attributes(G, 'compid').values()), key=lambda x: get_component_name(x))

cat_map = {k:i for i, k in enumerate(component_set)}
rev_cat_map ={i:k for i, k in enumerate(component_set)}


In [14]:
count_matrix = np.zeros((len(component_set), len(component_set)))


In [15]:

for edge in list(G.edges):
    source_id = edge[0]
    s_compid = G.nodes[source_id]['compid']
    recp_id = edge[1]
    r_compid = G.nodes[recp_id]['compid']
    count_matrix[cat_map[s_compid], cat_map[r_compid]] += 1


In [77]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

# Assuming count_matrix and cat_map are defined
# Normalize the matrix and set up the mask
count_matrix_nonzero = np.where(count_matrix == 0, 1e-10, count_matrix)
row_sums = count_matrix_nonzero.sum(axis=1, keepdims=True)
normalized_matrix = count_matrix_nonzero / row_sums
mask = normalized_matrix == 1e-10

# Define the axis labels using the 'cat_map' which maps component names to indices
axis_labels = [None] * len(cat_map)
for component, index in cat_map.items():
    axis_labels[index] = get_component_name(component)  # Make sure that the component names are placed according to their index



In [81]:
def get_top_connections(count_matrix, node_labels, top_n=20):
    top_connections = {}

    for i in range(len(count_matrix)):
        node = node_labels[i]
        total_count = sum(count_matrix[i])
        connections = [(j, count_matrix[i][j] / total_count) for j in range(len(count_matrix[i])) if count_matrix[i][j] > 0]
        connections.sort(key=lambda x: x[1], reverse=True)
        top_connections[node] = [(node_labels[j], round(probability * 100, 1)) for j, probability in connections[:top_n]]

    return top_connections
def print_component_connections(data):
    print("Component Connections and Probabilities:")
    max_connection_length = max(len(connection) for connections in data.values() for connection, _ in connections)
    
    for component, connections in data.items():
        print(f"{component}:")
        for connection, probability in connections:
            print(f"  - {connection:<{max_connection_length}}\t{probability:.1f}")
        print()  # Add an empty line between components


top_connections = get_top_connections(count_matrix, axis_labels)
print_component_connections(top_connections)

Component Connections and Probabilities:
Anemone:Fast Loop End:
  - Display:Custom Preview                          	12.5
  - Params:Curve                                    	12.5
  - Params:Panel                                    	12.5
  - Params:Geometry                                 	6.2
  - Params:Point                                    	6.2
  - Params:Surface                                  	6.2
  - Sets:Flatten Tree                               	6.2
  - Sets:Insert Items                               	6.2
  - Sets:List Length                                	6.2
  - Sets:Merge                                      	6.2
  - Surface:Boundary Surfaces                       	6.2
  - Surface:Brep Edges                              	6.2
  - Surface:Surface Closest Point                   	6.2

Anemone:Fast Loop Start:
  - Anemone:Fast Loop End                           	30.2
  - Sets:List Item                                  	14.0
  - Maths:Multiplication                          

In [83]:
# Generate incoming and outgoing connection probabilities
def get_connections(count_matrix, node_labels):
    top_connections = {}

    # Outgoing connections (as provided)
    for i in range(len(count_matrix)):
        node = node_labels[i]
        total_outgoing = sum(count_matrix[i])
        if total_outgoing > 0:
            outgoing_connections = [(node_labels[j], count_matrix[i][j] / total_outgoing)
                                    for j in range(len(count_matrix[i])) if count_matrix[i][j] > 0]
            top_connections[node] = {
                'outgoing': sorted(outgoing_connections, key=lambda x: x[1], reverse=True)
            }
        else:
            top_connections[node] = {'outgoing': []}
    
    # Incoming connections
    for j in range(len(count_matrix)):
        node = node_labels[j]
        total_incoming = sum(count_matrix[:,j])
        if total_incoming > 0:
            incoming_connections = [(node_labels[i], count_matrix[i][j] / total_incoming)
                                    for i in range(len(count_matrix)) if count_matrix[i][j] > 0]
            if node in top_connections:
                top_connections[node]['incoming'] = sorted(incoming_connections, key=lambda x: x[1], reverse=True)
            else:
                top_connections[node] = {'incoming': incoming_connections}

    return top_connections

def print_component_connections(data):
    print("Component Connections and Probabilities:")
    max_connection_length = max(len(connection) for connections in data.values() for direction in connections.values() for connection, _ in direction)
    
    for component, connections in data.items():
        print(f"{component}")
        # Zip the outgoing and incoming connections together
        combined_connections = zip(
            connections.get('outgoing', []),
            connections.get('incoming', [('', '')] * len(connections.get('outgoing', [])))  # Pad with empty tuples to ensure the lists have the same length
        )
        
        for (out_conn, out_prob), (in_conn, in_prob) in combined_connections:
            # Mirror the connection probabilities around the dash
            print(
                f"{out_prob:.1f}% {out_conn:<{max_connection_length}} | {in_conn:<{max_connection_length}} {in_prob:.2f}%"
            )
        print()  # Add an empty line between components

# Assume count_matrix and axis_labels are defined as per your graph G
top_connections = get_connections(count_matrix, axis_labels)
print_component_connections(top_connections)


Component Connections and Probabilities:
Anemone:Fast Loop End
0.1% Display:Custom Preview                           | Anemone:Fast Loop Start                          0.4%
0.1% Params:Curve                                     | Sets:Flatten Tree                                0.1%
0.1% Params:Panel                                     | Sets:Merge                                       0.1%
0.1% Params:Geometry                                  | Transform:Rotate                                 0.1%
0.1% Params:Point                                     | Curve:Curve Closest Point                        0.0%
0.1% Params:Surface                                   | Curve:Interpolate                                0.0%
0.1% Sets:Flatten Tree                                | Curve:Line                                       0.0%
0.1% Sets:Insert Items                                | Intersect:Solid Difference                       0.0%
0.1% Sets:List Length                                 | I

ValueError: Unknown format code 'f' for object of type 'str'

In [86]:
def get_top_connections(count_matrix, node_labels, top_n=20):
    top_incoming = {}
    top_outgoing = {}

    for i in range(len(count_matrix)):
        node = node_labels[i]
        total_incoming = sum(count_matrix[:, i])
        total_outgoing = sum(count_matrix[i, :])

        incoming_connections = [(j, count_matrix[j, i] / total_incoming) for j in range(len(count_matrix)) if count_matrix[j, i] > 0]
        outgoing_connections = [(j, count_matrix[i, j] / total_outgoing) for j in range(len(count_matrix)) if count_matrix[i, j] > 0]

        incoming_connections.sort(key=lambda x: x[1], reverse=True)
        outgoing_connections.sort(key=lambda x: x[1], reverse=True)

        top_incoming[node] = [(node_labels[j], round(probability * 100, 1)) for j, probability in incoming_connections[:top_n]]
        top_outgoing[node] = [(node_labels[j], round(probability * 100, 1)) for j, probability in outgoing_connections[:top_n]]

    return top_incoming, top_outgoing

def print_component_connections(incoming_data, outgoing_data):
    print("Component Connections and Probabilities:")
    max_incoming_length = max(len(connection) for connections in incoming_data.values() for connection, _ in connections)
    max_outgoing_length = max(len(connection) for connections in outgoing_data.values() for connection, _ in connections)

    for component in incoming_data:
        print(f"{component}:")
        incoming_connections = incoming_data[component]
        outgoing_connections = outgoing_data[component]

        max_connections = max(len(incoming_connections), len(outgoing_connections))

        for i in range(max_connections):
            incoming_connection = incoming_connections[i] if i < len(incoming_connections) else ("", "")
            outgoing_connection = outgoing_connections[i] if i < len(outgoing_connections) else ("", "")

            incoming_name, incoming_prob = incoming_connection
            outgoing_name, outgoing_prob = outgoing_connection

            incoming_prob_str = f"{incoming_prob:>5.1f}" if incoming_prob != "" else " " * 6
            outgoing_prob_str = f"{outgoing_prob:<5.1f}" if outgoing_prob != "" else " " * 6

            print(f"{incoming_prob_str}  {incoming_name:<{max_incoming_length}}  |  {outgoing_name:<{max_outgoing_length}}  {outgoing_prob_str}")

        print()  # Add an empty line between components

top_incoming, top_outgoing = get_top_connections(count_matrix, axis_labels)
print_component_connections(top_incoming, top_outgoing)

Component Connections and Probabilities:
Anemone:Fast Loop End:
 37.1  Anemone:Fast Loop Start                       |  Display:Custom Preview                            12.5 
  5.7  Sets:Flatten Tree                             |  Params:Curve                                      12.5 
  5.7  Sets:Merge                                    |  Params:Panel                                      12.5 
  5.7  Transform:Rotate                              |  Params:Geometry                                   6.2  
  2.9  Curve:Curve Closest Point                     |  Params:Point                                      6.2  
  2.9  Curve:Interpolate                             |  Params:Surface                                    6.2  
  2.9  Curve:Line                                    |  Sets:Flatten Tree                                 6.2  
  2.9  Intersect:Solid Difference                    |  Sets:Insert Items                                 6.2  
  2.9  Intersect:Solid Intersection     

In [104]:
import pandas as pd

def get_top_connections(count_matrix, node_labels, top_n=20):
    data = []

    for i in range(len(count_matrix)):
        node = node_labels[i]
        total_incoming = sum(count_matrix[:, i])
        total_outgoing = sum(count_matrix[i, :])

        incoming_connections = [(node_labels[j], count_matrix[j, i] / total_incoming) for j in range(len(count_matrix)) if count_matrix[j, i] > 0]
        outgoing_connections = [(node_labels[j], count_matrix[i, j] / total_outgoing) for j in range(len(count_matrix)) if count_matrix[i, j] > 0]

        incoming_connections.sort(key=lambda x: x[1], reverse=True)
        outgoing_connections.sort(key=lambda x: x[1], reverse=True)

        max_connections = max(len(incoming_connections), len(outgoing_connections))

        for k in range(max_connections):
            incoming_name, incoming_prob = incoming_connections[k] if k < len(incoming_connections) else ("", "")
            outgoing_name, outgoing_prob = outgoing_connections[k] if k < len(outgoing_connections) else ("", "")

            data.append({
                'Component': node if k == 0 else "",
                'Incoming Connection': incoming_name,
                'Incoming Probability': incoming_prob,
                'Outgoing Connection': outgoing_name,
                'Outgoing Probability': outgoing_prob
            })

    top_connections_df = pd.DataFrame(data)
    top_connections_df = top_connections_df.groupby('Component', sort=False).apply(lambda x: x.reset_index(drop=True))


    # Save the DataFrame to a CSV file
    top_connections_df.to_csv('top_connections.csv', index=False)

    return top_connections_df

top_connections_df = get_top_connections(count_matrix, axis_labels)
top_connections_df.head()

Component        Incoming Connection  \
Component                                                                   
Anemone:Fast Loop End 0  Anemone:Fast Loop End    Anemone:Fast Loop Start   
                      0                                 Sets:Flatten Tree   
                      1                                        Sets:Merge   
                      2                                  Transform:Rotate   
                      3                         Curve:Curve Closest Point   

                        Incoming Probability     Outgoing Connection  \
Component                                                              
Anemone:Fast Loop End 0             0.371429  Display:Custom Preview   
                      0             0.057143            Params:Curve   
                      1             0.057143            Params:Panel   
                      2             0.057143         Params:Geometry   
                      3             0.028571            Params:Point   

                        Outgoing Probability  
Component                                     
Anemone:Fast Loop End 0                0.125  
                      0                0.125  
                      1                0.125  
                      2               0.0625  
                      3               0.0625

In [67]:
import numpy as np
import holoviews as hv
import pandas as pd
import networkx as nx
from holoviews import opts, dim

hv.extension('matplotlib')

def get_component_name(id):
    try:
        p = GHComponentTable.search_component_by_guid(System.Guid(id))
        return f"{p.Desc.Category}:{p.Desc.Name}"
    except:
        return None

G = combined_graph

# Get the 'compid' attribute for each node in the graph
compid_dict = nx.get_node_attributes(G, 'compid')

# Filter out nodes with missing or invalid 'compid' attribute
valid_compids = {compid for compid in compid_dict.values() if compid is not None}

# Create the component set by sorting the valid 'compid' values
component_set = sorted(valid_compids, key=lambda x: get_component_name(x))

cat_map = {comp_id: i for i, comp_id in enumerate(component_set)}
rev_cat_map = {i: comp_id for i, comp_id in enumerate(component_set)}

count_matrix = np.zeros((len(component_set), len(component_set)))

for edge in G.edges:
    source_id, recp_id = edge
    s_compid = compid_dict.get(source_id)
    r_compid = compid_dict.get(recp_id)
    if s_compid in component_set and r_compid in component_set:
        count_matrix[cat_map[s_compid], cat_map[r_compid]] += 1

# Create a DataFrame from the count_matrix with axis_labels as index and columns
axis_labels = [get_component_name(comp_id) for comp_id in component_set if get_component_name(comp_id)]

df = pd.DataFrame(count_matrix, index=axis_labels, columns=axis_labels, dtype=int)
df.head()

edge_data = []
for i, rname in enumerate(df.index):
    for j, cname in enumerate(df.columns):
        v = df.iloc[i, j]
        edge_data.append((rname, cname, v))

# Create a Holoviews Dataset from the edge data
edges = hv.Dataset(edge_data, ['source', 'target'], 'value')

# Create a DataFrame with node labels and categories
categories = [label.split(":")[0] for label in axis_labels]
node_data = list(zip(axis_labels, categories))
nodes = hv.Dataset(node_data, ['label', 'category'])

# Create a Chord object
chord = hv.Chord((edges, nodes)).select(value=(5, None))

# Specify the options for the Chord object
chord.opts(
    opts.Chord(cmap='Category20', edge_color=dim('source').str(),
               labels='label', node_color=dim('category').str())
)

# Render the plot
hv.save(chord, 'chord_diagram.png', fmt='png')

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

IndexError: index 28 is out of bounds for axis 0 with size 28

Implemending the GNN model

In [18]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, LayerNorm, Sequential, Linear

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(ds.num_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.conv3 = GCNConv(16, 16)
        self.lns = LayerNorm(16)
        self.post_mp = torch.nn.Sequential(
            Linear(16, 16),
            torch.nn.Dropout(0.25),
            Linear(16, 2)
        )
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)
                # x= self.conv2(x, edge_index)
                # x = F.relu(x)
                # x = F.dropout(x, p=0.25, training=self.training)
                # x =self.lns(x)
                # x = self.conv3(x, edge_index)
                # x= F.relu(x)
                # x = F.dropout(x, p=0.25, training=self.training)
                # x = self.post_mp(x)
        return x, F.log_softmax(x, dim=1)
    


In [19]:
# combined_graph = nx.compose_all(graph_data[:2])
# for graph in graph_data[:2]:
#     plot_graph_by_category(graph)
# plot_graph_by_category(combined_graph)

In [20]:
# print(len(combined_graph))
# print(combined_graph.number_of_nodes())
# print(nx.number_connected_components(combined_graph))


In [21]:
data = [utils.from_networkx(G) for G in graph_data]
data[0]['edge_index']

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 10, 11, 12, 12, 13, 13, 14,
         15, 16, 17, 18, 19, 20, 21, 22, 22, 23, 24, 25, 25, 26, 27, 28, 29, 30,
         31, 31, 32, 33, 33, 34, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 43, 43,
         43, 44, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 55, 56, 58, 59,
         60],
        [ 1,  3,  0,  6,  1,  4,  9,  3,  3, 10, 11, 12, 13, 17, 19, 15, 16, 11,
         24, 26, 18, 22, 37, 17, 17, 24, 26, 18, 25, 28, 30, 27, 28, 29, 32, 29,
         25, 27, 33, 34, 35, 39, 40, 39, 19, 22, 32, 40, 43, 39, 41, 44, 45, 46,
         47, 48, 47, 49, 52, 55, 52, 50, 44, 44, 53, 54, 47, 56, 57, 58, 59, 57,
         59]])

In [22]:
plot = False
if plot:
    # Set the font size globally
    plt.rcParams.update({'font.size': 5})  
    plt.figure(figsize=(10,200))
    
    x = counts
    y = [f"{categories[i]}:{name}" for i, name in enumerate(names)]
    sns.barplot(x=x, y=y)
